In [1]:
import pandas as pd
import numpy as np


In [2]:
data = pd.read_csv("../Cleaning_Engineering/base_cleaned.csv")

In [3]:
data

,Unnamed: 0,distance,cab_type,source,destination,name,car_type,weekday,rush_hour,temp,clouds,pressure,rain,humidity,wind,is_raining,temp_groups,surge_multiplier,price
0,0,3.03,Lyft,Boston University,Theatre District,Lux Black XL,Luxury SUV,1,0,41.07,0.86,1014.39,NaN,0.92,1.36,0,40,1.0,34.0
1,1,1.30,Uber,South Station,Theatre District,Black,Luxury,1,0,40.86,0.87,1014.39,NaN,0.93,1.60,0,40,1.0,18.5
2,2,2.43,Lyft,Northeastern University,Beacon Hill,Lyft,Base,1,0,40.81,0.89,1014.35,NaN,0.93,1.36,0,40,1.0,10.5
3,3,2.71,Uber,Theatre District,Fenway,UberXL,Base XL,1,0,40.80,0.87,1014.39,NaN,0.93,1.55,0,40,1.0,32.0
4,4,2.71,Uber,Theatre District,Fenway,UberX,Base,1,0,40.80,0.87,1014.39,NaN,0.93,1.55,0,40,1.0,19.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637971,637971,2.40,Lyft,Northeastern University,Beacon Hill,Lyft XL,Base XL,1,0,30.91,0.00,1012.34,NaN,0.46,13.06,0,30,1.0,16.5
637972,637972,2.40,Lyft,Northeastern University,Beacon Hill,Lux Black XL,Luxury SUV,1,0,30.91,0.00,1012.34,NaN,0.46,13.06,0,30,1.0,32.5
637973,637973,2.40,Lyft,Northeastern University,Beacon Hill,Shared,Shared,1,0,30.91,0.00,1012.34,NaN,0.46,13.06,0,30,1.0,7.0
637974,637974,2.40,Lyft,Northeastern University,Beacon Hill,Lyft,Base,1,0,30.91,0.00,1012.34,NaN,0.46,13.06,0,30,1.0,10.5


In [4]:


# Assuming 'df' is your original DataFrame with the ride data
# Create empty DataFrames for eta_df and price_df
eta_df = pd.DataFrame()
price_df = pd.DataFrame()

# Calculate the base_price for each source-destination combination
base_price_series = data.groupby(['source', 'destination'])['price'].min()

# Define a function to apply the base price to the original DataFrame
def apply_base_price(row, base_price_series):
    return base_price_series.loc[row['source'], row['destination']]

# Apply the function and create the base_price column
data['base_price'] = data.apply(apply_base_price, axis=1, base_price_series=base_price_series)

# Split the data into two parts: 60% for price_df and 40% for eta_df
price_df = data.sample(frac=0.6, random_state=1)  # 60% sample
eta_df = data.drop(price_df.index)  # Remaining 40%

# Function to scale prices into a space between 0 and 1
def scale_prices(row, base_price_series):
    min_price = base_price_series.loc[row['source'], row['destination']]
    max_price = df.loc[
        (df['source'] == row['source']) & (df['destination'] == row['destination']),
        'price'
    ].max()
    # Prevent division by zero in case min_price equals max_price
    if min_price == max_price:
        return 1
    else:
        return (row['price'] - min_price) / (max_price - min_price)

# Apply the function and create the demand column for both DataFrames
price_df['demand'] = price_df.apply(scale_prices, axis=1, base_price_series=base_price_series)
eta_df['demand'] = eta_df.apply(scale_prices, axis=1, base_price_series=base_price_series)


NameError: name 'df' is not defined

In [ ]:
def data_transform(data):

    # Separate the features (X) and the target variable (y)
    X = data.drop('price', axis=1)
    y = data['price']

    # Preprocessing for numerical features
    numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]
    numerical_transformer = StandardScaler()

    # Preprocessing for categorical features
    categorical_cols = [cname for cname in X.columns if X[cname].dtype == 'object']
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')

    # Bundle preprocessing for numerical and categorical features
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)])

    # Split the dataset into training (80%) and validation (20%) sets
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

    # Preprocessing of training data, fit_transform
    X_train_prepared = preprocessor.fit_transform(X_train)
    
    # Preprocessing of validation data, transform
    X_valid_prepared = preprocessor.transform(X_valid)

    # Check the shape after preprocessing
    X_train_prepared.shape, X_valid_prepared.shape

    final_columns = preprocessor.get_feature_names_out()
    return (X_train_prepared, y_train, X_valid_prepared, y_valid, final_columns)


In [ ]:
data = pd.read_csv("../Cleaning_Engineering/base_cleaned.csv")

data = data.drop(columns=['Unnamed: 0',"name"])
data["rain"].fillna(0.0, inplace=True)
test_data = data.iloc[int(data.shape[0]*.9):]
data = data.iloc[:int(data.shape[0]*.9)]

user_data = data[["cab_type", "source", "destination", "car_type", "weekday", "rush_hour", "is_raining", "temp_groups", "surge_multiplier", "price"]]